In [1]:
import os
import sys
import boto3
from time import time
from time import sleep
import rasterio
from rasterio.plot import show
import xarray as xr
import rioxarray as rxr

In [2]:
#! free -h

In [3]:
#!pip install rioxarray --user
#!pip install rio-cogeo --user

In [4]:
def _xr_open_rasterio_retry(s3_file_name):
    cnt=20
    sleeptime=6
    while(cnt>0):
        try:
            da = rxr.open_rasterio(s3_file_name)
            print('SUCCESS _xr_open_rasterio_retry', s3_file_name, flush=True)
            return da
        except rasterio.errors.RasterioIOError:
                        print("Unexpected error:", sys.exc_info()[0])
                        print('oops',cnt)
                        print('oops',s3_file_name, flush=True)
                        cnt = cnt - 1
                        sleep(sleeptime)

In [5]:
def xr_build_mosaic_ds(bucket, product, tifs):

    start = time()
    my_da_list =[]
    for tif in tifs:
        try:
            da = _xr_open_rasterio_retry(f's3://{bucket}/'+tif)
        except:
            print('error on ', tif, flush=True)
            print('FATAL error on ', tif, flush=True)

        try:
            da = da.squeeze().drop(labels='band')
            da.name=product
            my_da_list.append(da)
            tnow = time()
            elapsed = tnow - start
            print(tif, elapsed)
        except:
            print('FATAL SQUEEZE error on ', tif, flush=True)

    try:
        DS = xr.merge(my_da_list)
        return(DS)
    except:
        print('FATAL MERGE error on ', tif, flush=True)

In [6]:
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        out_bucket = return_my_bucket(bucket_filepath)
        a = bucket_filepath.split('/')
        bucket_filepath = '/'.join(a[1:]) # strip bucket from path
        print('PUSH', out_bucket, bucket_filepath)
        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            s3.upload_fileobj(f, out_bucket, bucket_filepath)
        os.remove(local_file)

In [7]:
def _run_command(cmd, verbose=False):
    if verbose:
        print(cmd)
    result = os.system(cmd)
    if result != 0:
        raise Exception('command "%s" failed with code %d.' % (cmd, result))

In [8]:
def cog_create_from_tif(src_tif,dst_cog):
    command = f'rio cogeo create {src_tif} {dst_cog}'
    _run_command(command)

In [9]:
def return_my_bucket(prefix_with_slash):
    a = prefix_with_slash.split('/')
    print('a=',a)
    THE_BUCKET=a[0]
    print("the BUCKET=",THE_BUCKET)
    return THE_BUCKET

In [10]:
def xr_write_geotiff_from_ds(DS, primary_name, out_prefix_path):
    
    print(DS) # DS is the xarray
    print(primary_name) # first tif 
    print(out_prefix_path)

    a = primary_name.split('/')
    just_tif = a[-2] + '/' + a[-1]
    local_tif = a[-1]
    local_cog = 'COG_' + local_tif

    output = out_prefix_path + just_tif
    bucket = 'ws-enduser'
    print(f'OUTPUT=={output}')
    DS.rio.to_raster(local_tif)
    cog_create_from_tif(local_tif, local_cog)
    s3_push_delete_local(local_cog, bucket, output)
    os.remove(local_tif)
    #local_xml = local_cog + '.aux.xml'
    #os.remove(local_xml)

In [11]:
bucket = 'ws-enduser'
product = 'etasw'
year = '2000'
day = '12'

tifs = [f'CONUS/r37.0_tile0/{year}/{product}_{year}{day}.tif',
        f'CONUS/r37.0_tile1/{year}/{product}_{year}{day}.tif',
        f'CONUS/r37.0_tile2/{year}/{product}_{year}{day}.tif',
        f'CONUS/r37.0_tile3/{year}/{product}_{year}{day}.tif',
        f'CONUS/r50.0_tile5/{year}/{product}_{year}{day}.tif',
        f'CONUS/r50.0_tile6/{year}/{product}_{year}{day}.tif',
        f'CONUS/r50.0_tile7/{year}/{product}_{year}{day}.tif',
        f'CONUS/r50.0_tile8/{year}/{product}_{year}{day}.tif',
        f'CONUS/r50.0_tile9/{year}/{product}_{year}{day}.tif']

In [12]:
# s3_file_name = 's3://ws-enduser/CONUS/r37.0_tile0/2000/etasw_200001.tif'
# da = xr.open_rasterio(s3_file_name)
# da

In [13]:
DS = xr_build_mosaic_ds(bucket, product, tifs)

SUCCESS _xr_open_rasterio_retry s3://ws-enduser/CONUS/r37.0_tile0/2000/etasw_200012.tif
CONUS/r37.0_tile0/2000/etasw_200012.tif 0.5806293487548828
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/CONUS/r37.0_tile1/2000/etasw_200012.tif
CONUS/r37.0_tile1/2000/etasw_200012.tif 0.8763687610626221
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/CONUS/r37.0_tile2/2000/etasw_200012.tif
CONUS/r37.0_tile2/2000/etasw_200012.tif 1.1119680404663086
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/CONUS/r37.0_tile3/2000/etasw_200012.tif
CONUS/r37.0_tile3/2000/etasw_200012.tif 1.3807201385498047
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/CONUS/r50.0_tile5/2000/etasw_200012.tif
CONUS/r50.0_tile5/2000/etasw_200012.tif 1.711730718612671
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/CONUS/r50.0_tile6/2000/etasw_200012.tif
CONUS/r50.0_tile6/2000/etasw_200012.tif 2.016648530960083
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/CONUS/r50.0_tile7/2000/etasw_200012.tif
CONUS/r50.0_tile7/2000/etasw_200

In [14]:
DS

<xarray.Dataset>
Dimensions:      (x: 31249, y: 12978)
Coordinates:
  * x            (x) float64 -125.0 -125.0 -125.0 -125.0 ... -60.0 -60.0 -60.0
  * y            (y) float64 23.0 23.0 23.0 23.01 23.01 ... 49.99 50.0 50.0 50.0
    spatial_ref  int64 0
Data variables:
    etasw        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    _FillValue:    -9999.0
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     etasw_

In [15]:
# DS = xr_build_mosaic_ds(bucket, product, tifs)
primary_name = tifs[0]  #first tif in list
print(primary_name)
out_prefix_path = 'ws-enduser/CONUS/conus_mos/' # it gets the "year" folder from the function

xr_write_geotiff_from_ds(DS, primary_name, out_prefix_path)

CONUS/r37.0_tile0/2000/etasw_200012.tif
<xarray.Dataset>
Dimensions:      (x: 31249, y: 12978)
Coordinates:
  * x            (x) float64 -125.0 -125.0 -125.0 -125.0 ... -60.0 -60.0 -60.0
  * y            (y) float64 23.0 23.0 23.0 23.01 23.01 ... 49.99 50.0 50.0 50.0
    spatial_ref  int64 0
Data variables:
    etasw        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    _FillValue:    -9999.0
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     etasw_
CONUS/r37.0_tile0/2000/etasw_200012.tif
ws-enduser/CONUS/conus_mos/
OUTPUT==ws-enduser/CONUS/conus_mos/2000/etasw_200012.tif


Reading input: /home/ubuntu/opt/etm_study/etasw_200012.tif

Adding overviews...
Updating dataset tags...
Writing output to: /home/ubuntu/opt/etm_study/COG_etasw_200012.tif


a= ['ws-enduser', 'CONUS', 'conus_mos', '2000', 'etasw_200012.tif']
the BUCKET= ws-enduser
PUSH ws-enduser CONUS/conus_mos/2000/etasw_200012.tif


In [16]:
#! aws s3 ls s3://ws-enduser/CONUS/conus_mos/2000/

In [17]:
#! rio info s3://ws-enduser/CONUS/conus_mos/2000/etasw_200002.tif

In [18]:
#! rio info s3://ws-enduser/CONUS/conus_mos/2000/etasw_200002.tif | python -m json.tool

In [19]:
#! gdalinfo /vsis3/ws-enduser/USA/conus_mos/2000/2000/etasw_200001.tif - not working yet

### open geoTIFF from an amazon data bucket

In [20]:
#s3_file = 's3://ws-enduser/CONUS/conus_mos/2000/etasw_200002.tif'

In [21]:
#with rasterio.open(s3_file) as src:
#    print(src.profile)

In [22]:
#with rasterio.open(s3_file) as src:
#    print(src.profile)
##    np.flip(axis=1)
#    show(src)
#    array=src.read(1)
#print(array)